<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/projetoComCSV/teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#uploading files
from google.colab import files
uploaded = files.upload()

#importing libs
import pandas as pd

Saving alzheimer.csv to alzheimer (1).csv


In [ ]:

# importing multiclass dataset
dfm = pd.read_csv('obesity.csv')

#changing columns names
# Defining a dictionary with old column names as keys and new column names as values
column_mapping = {
    'family_history_with_overweight': 'family_overweight',
    'FAVC': 'caloric_food',
    'FCVC': 'eat_vegetables',
    'NCP': 'meals_day',
    'SMOKE': 'smoke',
    'CH2O': 'water_day',
    'SCC': 'track_calories',
    'FAF': 'physical_activity_week',
    'CALC': 'alcohol',
    'MTRANS': 'transportation_method',
    'NObeyesdad': 'obesity_level'
}

# Renaming the columns using the dictionary
dfm = dfm.rename(columns=column_mapping)


#Removing unnecessary columns
dfm = dfm.drop(columns=['CAEC', 'TUE'])

#encoding values (binary)
binary_mappings = {
    "Gender": {"Female": 0, "Male": 1},
    "family_overweight": {"yes": 1, "no": 0},
    "caloric_food": {"yes": 1, "no": 0},
    "smoke": {"yes": 1, "no": 0},
    "track_calories": {"yes": 1, "no": 0}
}

for col, mapping in binary_mappings.items():
    dfm[col] = dfm[col].map(mapping)

#encoding alcohol usage
encoding_map = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dfm['alcohol'] = dfm['alcohol'].map(encoding_map)


#encoding transportation_method
custom_mapping = {
    "Walking": 0,
    "Bike": 0,
    "Motorbike": 1,
    "Automobile": 1,
    "Public_Transportation": 1
}
dfm['transportation_method'] = dfm['transportation_method'].map(custom_mapping)

#encoding output column (obesity_level)
obesity_mapping = {
    "Insufficient_Weight": 0,
    "Normal_Weight": 1,
    "Overweight_Level_I": 2,
    "Overweight_Level_II": 3,
    "Obesity_Type_I": 4,
    "Obesity_Type_II": 5,
    "Obesity_Type_III": 6
}
dfm['obesity_level'] = dfm ['obesity_level'].map(obesity_mapping)

#normalizing data
columns_to_normalize = ["Height", "Weight", "Age"]
dfm[columns_to_normalize] = dfm[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

#spliting the dataframe
#training 80% of dataset
#testing 20% of dataset
split_point = int(0.8 * len(dfm))
train_multiclass = dfm[:split_point]
test_multiclass = dfm[split_point:]


dfm.head(10)

,Gender,Age,Height,Weight,family_overweight,caloric_food,eat_vegetables,meals_day,smoke,water_day,track_calories,physical_activity_week,alcohol,transportation_method,obesity_level
0,0,0.149,0.321,0.187,1,0,2.0,3.0,0,2.0,0,0.0,0,1,1
1,0,0.149,0.132,0.127,1,0,3.0,3.0,1,3.0,1,3.0,1,1,1
2,1,0.191,0.660,0.284,1,0,2.0,3.0,0,2.0,0,2.0,2,1,1
3,1,0.277,0.660,0.358,0,0,3.0,3.0,0,2.0,0,2.0,2,0,2
4,1,0.170,0.623,0.379,0,0,2.0,1.0,0,2.0,0,0.0,1,1,3
5,1,0.319,0.321,0.104,0,1,2.0,3.0,0,2.0,0,0.0,1,1,1
6,0,0.191,0.094,0.119,1,1,3.0,3.0,0,2.0,0,1.0,1,1,1
7,1,0.170,0.358,0.104,0,0,2.0,3.0,0,2.0,0,3.0,1,1,1
8,1,0.213,0.623,0.187,1,1,3.0,3.0,0,2.0,0,1.0,2,1,1
9,1,0.170,0.509,0.216,1,1,2.0,3.0,0,2.0,0,1.0,0,1,1


In [91]:
import numpy as np
import pandas as pd

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))

# Xavier initialization
def xavier_initialization(input_dim, output_dim):
    return np.random.randn(input_dim, output_dim) * np.sqrt(2. / (input_dim + output_dim))

# Load and preprocess the dataset
dfb = pd.read_csv('alzheimer.csv')

# Selecting useful columns
selected_columns = ['Age', 'Gender', 'Smoking', 'EducationLevel', 'BMI', 'SleepQuality',
                    'CholesterolLDL', 'FamilyHistoryAlzheimers', 'PhysicalActivity', 'Diagnosis']
dfb = dfb[selected_columns]

# Normalizing data
columns_to_normalize = ["BMI", "SleepQuality", "Age", "CholesterolLDL", "FamilyHistoryAlzheimers", "PhysicalActivity"]
dfb[columns_to_normalize] = dfb[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

random_number = random.randint(1, 100)
# Shuffle the entire dataset (80% train, 20% test)
dfb = dfb.sample(frac=1, random_state=random_number).reset_index(drop=True)  # Shuffle all rows

# Split dataset (80% train, 20% test)
split_point2 = int(0.8 * len(dfb))
train_binary = dfb[:split_point2]
test_binary = dfb[split_point2:]



# Prepare data
X_train = train_binary.iloc[:, :-1].values  # Features
y_train = train_binary['Diagnosis'].values.reshape(-1, 1)  # Labels

X_test = test_binary.iloc[:, :-1].values
y_test = test_binary['Diagnosis'].values.reshape(-1, 1)

# Define neural network structure
input_size = X_train.shape[1]  # Number of features
hidden_size = 10               # Number of neurons in the first hidden layer
hidden_size2 = 6               # Number of neurons in the second hidden layer
output_size = 1                # Binary classification (single neuron)

# Initialize weights using Xavier initialization
np.random.seed(41)
W1 = xavier_initialization(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = xavier_initialization(hidden_size, hidden_size2)
b2 = np.zeros((1, hidden_size2))
W3 = xavier_initialization(hidden_size2, output_size)
b3 = np.zeros((1, output_size))

learning_rate = 0.01  # Lower learning rate
patience = 50
best_loss = float('inf')
no_improve_epochs = 0

# Training loop
for epoch in range(2000):
    # Forward propagation
    Z1 = np.dot(X_train, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = relu(Z2)  # Changed to relu
    Z3 = np.dot(A2, W3) + b3
    A3 = sigmoid(Z3)

    # Compute loss
    loss = binary_cross_entropy(y_train, A3)

    # Early stopping
    if loss < best_loss:
        best_loss = loss
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

    # Backpropagation
    dZ3 = A3 - y_train
    dW3 = np.dot(A2.T, dZ3) / X_train.shape[0]
    db3 = np.sum(dZ3, axis=0, keepdims=True) / X_train.shape[0]

    dA2 = np.dot(dZ3, W3.T)
    dZ2 = dA2 * relu_derivative(Z2)  # Changed to relu_derivative
    dW2 = np.dot(A1.T, dZ2) / X_train.shape[0]
    db2 = np.sum(dZ2, axis=0, keepdims=True) / X_train.shape[0]

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X_train.T, dZ1) / X_train.shape[0]
    db1 = np.sum(dZ1, axis=0, keepdims=True) / X_train.shape[0]

    # Update weights
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Predict on test set
Z1_test = np.dot(X_test, W1) + b1
A1_test = relu(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
A2_test = relu(Z2_test)
Z3_test = np.dot(A2_test, W3) + b3
A3_test = sigmoid(Z3_test)

# Make predictions by thresholding the output (class 0 or 1)
y_pred = (A3_test > 0.5).astype(int)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Diagnosis
0    283
1    147
Name: count, dtype: int64
Epoch 0, Loss: 0.6898
Epoch 100, Loss: 0.6655
Epoch 200, Loss: 0.6557
Epoch 300, Loss: 0.6521
Epoch 400, Loss: 0.6509
Epoch 500, Loss: 0.6504
Epoch 600, Loss: 0.6502
Epoch 700, Loss: 0.6500
Epoch 800, Loss: 0.6499
Epoch 900, Loss: 0.6497
Epoch 1000, Loss: 0.6496
Epoch 1100, Loss: 0.6495
Epoch 1200, Loss: 0.6494
Epoch 1300, Loss: 0.6493
Epoch 1400, Loss: 0.6492
Epoch 1500, Loss: 0.6491
Epoch 1600, Loss: 0.6490
Epoch 1700, Loss: 0.6489
Epoch 1800, Loss: 0.6488
Epoch 1900, Loss: 0.6487
Test Accuracy: 0.6581


In [ ]:
#import regression dataset
dfr = pd.read_csv('houses.csv')

#removing columns
dfr = dfr.drop(columns=['renovated', 'quartile_zone'])

#encoding true/false
dfr = dfr.replace({True: 1, False: 0})

#Extracting the year from the data
# Convert 'date' column to datetime format
dfr['date'] = pd.to_datetime(dfr['date'])
# Extract the year and replace the original 'date' column
dfr['date'] = dfr['date'].dt.year

#normalizing data
columns_to_normalize = ["price", "living_in_m2"]
dfr[columns_to_normalize] = dfr[columns_to_normalize].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)


#spliting the dataframe
#training 80% of dataset
#testing 20% of dataset
split_point3 = int(0.8 * len(dfr))
train_regression = dfr[:split_point3]
test_regression = dfr[split_point3:]

train_regression.head(10)

<ipython-input-18-1c407e60d625>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfr = dfr.replace({True: 1, False: 0})


,date,price,bedrooms,grade,has_basement,living_in_m2,nice_view,perfect_condition,real_bathrooms,has_lavatory,single_floor,month
0,2014,0.215,2,2,1,0.261,0,0,2,1,0,5
1,2014,0.213,2,2,0,0.166,0,0,1,1,0,11
2,2014,0.223,2,2,0,0.201,1,0,1,0,1,12
3,2015,0.171,2,3,0,0.299,0,0,2,1,1,2
4,2015,0.576,3,2,1,0.856,0,0,3,0,0,1
5,2015,0.339,1,2,1,0.103,1,0,1,0,1,3
6,2014,0.149,2,2,1,0.253,0,0,1,1,1,6
7,2014,0.143,1,1,0,0.264,0,0,1,0,1,10
8,2014,0.116,2,2,0,0.166,0,0,1,1,1,7
9,2014,0.432,2,3,0,0.204,0,0,2,1,0,9
